In [2]:
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.ad import Ad
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.ad import Ad

import pandas as pd

In [4]:
import sys
sys.path.append('../')

from utils.utils import date_validation

In [14]:
Ad._field_types

{'account_id': 'string',
 'ad_review_feedback': 'AdgroupReviewFeedback',
 'adlabels': 'list<AdLabel>',
 'adset': 'AdSet',
 'adset_id': 'string',
 'bid_amount': 'int',
 'bid_info': 'map<string, unsigned int>',
 'bid_type': 'BidType',
 'campaign': 'Campaign',
 'campaign_id': 'string',
 'configured_status': 'ConfiguredStatus',
 'conversion_domain': 'string',
 'conversion_specs': 'list<ConversionActionQuery>',
 'created_time': 'datetime',
 'creative': 'AdCreative',
 'demolink_hash': 'string',
 'display_sequence': 'int',
 'effective_status': 'EffectiveStatus',
 'engagement_audience': 'bool',
 'failed_delivery_checks': 'list<DeliveryCheck>',
 'id': 'string',
 'issues_info': 'list<AdgroupIssuesInfo>',
 'last_updated_by_app_id': 'string',
 'name': 'string',
 'preview_shareable_link': 'string',
 'priority': 'unsigned int',
 'recommendations': 'list<AdRecommendation>',
 'source_ad': 'Ad',
 'source_ad_id': 'string',
 'status': 'Status',
 'targeting': 'Targeting',
 'tracking_and_conversion_with_de

In [ ]:
Campaign._field_types

In [125]:
APP_ID = '1087573895408523'
APP_SECRET = '6e01d697a935b056a982ef623fde14e2'
ACCESS_TOKEN = 'EAAPdJIez64sBAHbj6nbhZALnueLwaWqZBAtCLZCZApDovwmbA2SjnYChZAMuyvMkfz2OHI8rMRNQcJJwv6b1yNuxkaqQXZCtV8T114uZCspIjGLFRZCzyov6fMoFQ5SJRkbmZC9Kx7r7QVGPA5vhu61SS18bZBJeZBZCKewunVRWpvKXc8v1qSu0KDY6XyBvURUi7kx4ReZA589BIMtW9LIiKZCvMHp8GEij9hKo0QVqrNL3iNYUz2s85VxDvG'
ACCOUNT_ID = '388606526375999'

In [126]:
FacebookAdsApi.init(APP_ID, APP_SECRET, ACCESS_TOKEN, api_version='v12.0')

In [127]:
account = AdAccount('act_' + ACCOUNT_ID)


In [115]:
FIELDS = ['adset_id', 'ad_name', 'campaign_id', 'created_time']

In [149]:
def get_ad_report(ad_account: AdAccount) -> pd.DataFrame:
    fields = ('id', 'adset_id', 'name', 'campaign_id',)

    ads = ad_account.get_ads(fields=fields)
    data = list(map(lambda ad: ad.export_all_data(), ads))

    return pd.DataFrame(data).rename(columns={'id': 'ad_id', 'name': 'ad_name'})



In [150]:
get_ad_report(account)

,ad_id,adset_id,ad_name,campaign_id
0,23849112002430765,23849112002390765,Продвижение веб-сайта: https://meow.com/,23849112002340765


In [172]:
def get_account_name(account_id: str) -> str:
    return AdAccount('act_' + account_id).api_get(fields=('name',)).get('name')

In [11]:
def get_campaign_report(ad_account: AdAccount) -> pd.DataFrame:
    fields = ('id', 'name', 'buying_type', 'smart_promotion_type', 'status', 'account_id')

    campaigns = ad_account.get_campaigns(fields=fields)
    data = list(map(lambda campaign: campaign.export_all_data(), campaigns))
    for row in data:
        row['account_name'] = get_account_name(row['account_id'])

    return pd.DataFrame(data).rename(columns={'id': 'campaign_id', 'name': 'campaign_name'})

In [177]:
get_campaign_report(account)


,campaign_id,campaign_name,buying_type,smart_promotion_type,status,account_id,account_name
0,23849112002340765,[19.12.2021] Продвижение https://meow.com/,AUCTION,GUIDED_CREATION,ACTIVE,388606526375999,Максим Холькин


In [115]:
def get_stat_report(ad_account: AdAccount, date_from: str, date_to: str) -> pd.DataFrame:
    fields = ('campaign_id', 'ad_id', 'adset_id', 'impressions', 'clicks', 'spend', 'account_currency',)
    params = {'time_range': {'since': date_from, 'until': date_to}, 'time_increment': 1}

    data = []
    ads = ad_account.get_ads()
    for ad in ads:
        insights = ad.get_insights(fields=fields, params=params)
        data += list(map(lambda insight: insight.export_all_data(), insights))
    
    return pd.DataFrame(data).drop(columns=['date_stop']).rename(columns={'date_start': 'dt', 'account_currency': 'currency'})

In [116]:
get_stat_report(account, '2021-12-18', '2022-01-10')

,campaign_id,ad_id,adset_id,impressions,clicks,spend,currency,dt
0,23849112002340765,23849112002430765,23849112002390765,205,3,53.77,RUB,2021-12-19
1,23849112002340765,23849112002430765,23849112002390765,306,4,97.53,RUB,2021-12-20
2,23849112002340765,23849112002430765,23849112002390765,284,5,98.87,RUB,2021-12-21
3,23849112002340765,23849112002430765,23849112002390765,246,5,99.6,RUB,2021-12-22
4,23849112002340765,23849112002430765,23849112002390765,279,3,98.34,RUB,2021-12-23
5,23849112002340765,23849112002430765,23849112002390765,271,6,100.74,RUB,2021-12-24
6,23849112002340765,23849112002430765,23849112002390765,299,10,97.25,RUB,2021-12-25
7,23849112002340765,23849112002430765,23849112002390765,381,2,107.78,RUB,2021-12-26
8,23849112002340765,23849112002430765,23849112002390765,421,3,100.37,RUB,2021-12-27
9,23849112002340765,23849112002430765,23849112002390765,392,2,110.2,RUB,2021-12-28


In [129]:
account.get_targeting_sentence_lines()

FacebookRequestError: 

  Message: Call was not successful
  Method:  GET
  Path:    https://graph.facebook.com/v12.0/act_388606526375999/targetingsentencelines
  Params:  {'summary': 'true'}

  Status:  400
  Response:
    {
      "error": {
        "message": "(#100) For field 'targetingsentencelines': The parameter targeting_spec is required",
        "type": "OAuthException",
        "code": 100,
        "fbtrace_id": "AGbi6HU8xbwirlZdKgFS4zD"
      }
    }
